In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import sbi

In [2]:
covariance = np.loadtxt("input_data/covariance_cosmic_shear_PMEpaper.dat")
precision  = np.linalg.inv(np.matrix(covariance))
N_data = len(covariance[:,0])
fiducial_dv = np.loadtxt("input_data/DES_shear-shear_a1.0_b0.5_data_vector")[:,1]
derivatives = np.loadtxt("input_data/derivatives.dat")
Omega_m_fid = 0.3156
sigma_8_fid = 0.831
w0_fid      = -1.0

In [3]:
Fisher_matrix = np.zeros((3,3))
for i in range(0,3):
    for j in range(0,3):
       Fisher_matrix[i,j] = np.linalg.multi_dot([derivatives[:,j], precision, derivatives[:,i]])

print(Fisher_matrix)
print(np.linalg.inv(np.matrix(Fisher_matrix)))

np.savetxt("Fisher_matrix.dat", Fisher_matrix)

print(np.sqrt(np.diag(np.linalg.inv(np.matrix(Fisher_matrix)))))

[[317652.9079044  213644.33873155 -13907.62322598]
 [213644.33873155 149360.77671246  -9873.38289421]
 [-13907.62322598  -9873.38289421    761.92298511]]
[[ 8.59540562e-05 -1.34138903e-04 -1.69292859e-04]
 [-1.34138903e-04  2.56028599e-04  8.69267734e-04]
 [-1.69292859e-04  8.69267734e-04  9.48672244e-03]]
[0.00927114 0.01600089 0.09739981]


In [4]:
# linearize model for the data vector
def linearized_model(params):
    dOm = params[0]-Omega_m_fid
    ds8 = params[1]-sigma_8_fid
    dw  = params[2]-w0_fid
    return fiducial_dv + derivatives[:,0]*dOm + derivatives[:,1]*ds8 + derivatives[:,2]*dw

# treat this as a black box with which you
# generate your simulated data vectors
def simulator(params, seed):
    expec = linearized_model(params)
    np.random.seed(seed)
    return np.random.multivariate_normal(expec, covariance)